# COGS 108 - Assignment 3: Data Analysis

This assignment covers extensive data cleaning, EDA, and regression.

This assignment is out of 8 points, worth 8% of your grade.

**PLEASE DO NOT CHANGE THE NAME OF THIS FILE.**

**PLEASE DO NOT COPY & PASTE OR DELETE CELLS INLCUDED IN THE ASSIGNMENT.** (Note that you *can* add additional cells, if you want to test things out.)

# Important Reminders

- This assignment has hidden tests: tests that are not visible here, but that will be run on your submitted assignment for grading.
    - This means passing all the tests you can see in the notebook here does not guarantee you have the right answer!
    - In particular many of the tests you can see simply check that the right variable names exist. Hidden tests check the actual values. 
        - It is up to you to check the values, and make sure they seem reasonable.
- A reminder to restart the kernel and re-run the code as a first line check if things seem to go weird.
    - For example, note that some cells can only be run once, because they re-write a variable (for example, your dataframe), and change it in a way that means a second execution will fail. 
    - Also, running some cells out of order might change the dataframe in ways that may cause an error, which can be fixed by re-running.

Run the following cell. These are all you need for the assignment. Do not import additional packages.

In [1]:
# Imports 
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
sns.set()
sns.set_context('talk')

import warnings
warnings.filterwarnings('ignore')

import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest
# Note: the statsmodels import may print out a 'FutureWarning'. Thats fine.

ModuleNotFoundError: No module named 'patsy'

## Setup

Data: the responses collected from a previous survery of the COGS 108 class. 
- There are 416 observations in the data, covering 10 different 'features'.

Research Question: Do students in different majors have different heights?

Background: Physical height has previously shown to correlate with career choice, and career success. More recently it has been demonstrated that these correlations can actually be explained by height in high school, as opposed to height in adulthood (1). It is currently unclear whether height correlates with choice of major in university. 

Reference: 1) https://www.sas.upenn.edu/~apostlew/paper/pdf/short.pdf

Hypothesis: We hypothesize that there will be a relation between height and chosen major. 

## Part 1: Load & Clean the Data (2.5 points)


Fixing messy data makes up a large amount of the work of being a Data Scientist. 

The real world produces messy measurements and it is your job to find ways to standardize your data such that you can make useful analyses out of it. 

In this section, you will learn, and practice, how to successfully deal with unclean data.

### 1a) Load the data
Import datafile `COGS108_IntroQuestionnaireData.csv` into a DataFrame called `df`.

In [2]:
# YOUR CODE HERE
df = pd.read_csv('COGS108_IntroQuestionnaireData.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'COGS108_IntroQuestionnaireData.csv'

In [3]:
assert isinstance(df, pd.DataFrame)


NameError: name 'df' is not defined

In [4]:
# Check out the data
df.head(5)

NameError: name 'df' is not defined

Those column names are a bit excessive, so first let's rename them - code provided below to do so. 

In [5]:
# Renaming the columns of the dataframe
df.columns = ['timestamp', 'year', 'major', 'age', 'gender', 'height',
              'weight', 'eye_color', 'born_in_CA', 'favorite_icecream']

NameError: name 'df' is not defined

`pandas` has a very useful function for detecting missing data. This function is called `isnull()`. 

If you have a dataframe called `df`, then calling `df.isnull()` will return another dataframe of the same size as `df` where every cell is either True of False. 

Each True or False is the answer to the question 'is the data in this cell null?'. So, False, means the cell is not null (and therefore, does have data). True means the cell is null (does not have data).

This function is very useful because it allows us to find missing data very quickly in our dataframe. As an example, consider the code below.

In [6]:
# Check the first few rows of the 'isnull' dataframe
df.isnull().head(5)

NameError: name 'df' is not defined

If you print out more, and scroll down, you'll see some rows with missing data. 

In [7]:
# For example:
df.isnull().iloc[48:50, :]

NameError: name 'df' is not defined

 Check an example, row 49, in which an entry has missing data:

In [8]:
df.iloc[49, :]

NameError: name 'df' is not defined

Granted, the example above is not very informative. As you can see, the output of `isnull()` is a dataframe where the values at each cell is either True or False. Most cells have the value of `False`. We expect this to be the case since most people gave out answers to each question in our survey. 

However, some rows such as row 49 show that some people chose not to answer certain questions. In the case of row 49, it seems that someone did not give out an answer for 'What year (in school) are you?'

However, what if we wanted to use `isnull()` to see all rows where our dataframe `df` has missing values? In other words, what if we want to see the ACTUAL rows with missing values instead of this dataframe with True or False cells. For that, we need to write the following line of code:
<br>
<br>
`df[df.isnull().any(axis=1)]`

### 1b) Find missing data
Find all rows that have missing data in them.

Save the ouput, as a dataframe, into a variable called `rows_to_drop`.

In other words, copy over and use the line of code that we gave out in the cell above.

In [9]:
# YOUR CODE HERE
rows_to_drop = df[df.isnull().any(axis=1)]

NameError: name 'df' is not defined

In [10]:
# check output
rows_to_drop.shape

NameError: name 'rows_to_drop' is not defined

In [11]:
assert isinstance(rows_to_drop, pd.DataFrame)
assert rows_to_drop.shape == (31, 10)

NameError: name 'rows_to_drop' is not defined

In the cell below, briefly explain below how `df[df.isnull().any(axis=1)]` works, in a couple sentences. 

Include an explanation of what `any(axis=1)` means and how it affects the code.

df[df.isnull().any(axis=1)] works by filtering the DataFrame to only include the rows that have one or more missing values. any(axis=1) is applied to the DataFrame to collapse each row by applying 'any' function to return 'True'. axis=1, when 'True', specifies that the method should apply horizontally.

Run the following cell and look at its output, but you don't need to add any code here.

Real world data are messy. As an example of it, we consider the data shown in `rows_to_drop` (below).

If you've done everything correctly so far, you should see an unexpected response with emojis at index 357. These types of responses, although funny, are hard to parse when dealing with big datasets. 

We'll learn about solutions to these types of problems in the upcoming cells.

In [12]:
rows_to_drop

NameError: name 'rows_to_drop' is not defined

### 1c) Drop the rows with NaN values

Drop any rows with missing data, but only for the columns `major`, `height`, `gender` and `age`. These will be the data of primary interest for our analyses, so we drop missing data here. 

Note that there are other missing data (in other rows) but this is fine for our analyses, so we keep them. 

To do this, use the pandas `dropna` method, inplace, using the `subset` arguments to specify columns.

In [13]:
# YOUR CODE HERE
df.dropna(subset=['major', 'height', 'gender', 'age'], inplace=True)

NameError: name 'df' is not defined

In [14]:
assert df.shape == (403, 10)

NameError: name 'df' is not defined

In the rest of Part 1, we will work on writing code, organized into functions that will allow us to transform similar respones into the same value. We will call this process: standardizing the data. 

The cell below provides an example for the kind of code you will need to write to answer this question. This example is separate from our actual data, and is a potential function we might use to standardize messy data - in this case, hypothetical data to the question 'What is your favourite major python version?'. 

Note some things used in this example that you need to use to standardize data:
- string methods, such as `lower` and `strip` to transform strings
- the `replace` string method, to replace a set of characters with something else
- if/else statements that check what's in our string (number, letters, etc)
- type casting, for example using `int()` to turn a variable into an integer
- using `np.nan` (which stands for 'not a number') to denote missing or unknown data

**Note**: For the following few cells you should run, read, and understand the code provided, but you don't have to add any code until Q1d.

The line of code above shows us the different values we got, to the question 'What year (in school) are you?'.

As you can tell, it is a <b>mess</b>!. For example, if you are a junior student, then you might have answered: 3, three, third, 3rd year, junior, junior year, Junior, etc. 

That is an issue. We want to be able to analyze this data and, in order to do this successfully, we need to all answers with the same meaning to be written in the same way. Therefore, we're gonna have to transform answers such as '3, third, 3rd, junior, etc' into a single possible value. We'll do this for all values that mean the same. 

In [15]:
def example_standardize_function(str_in):
    '''Standardize data to the question 'what is your favorite major python version?'
    
    Parameters
    ----------
    str_in : string
        A provided answer.
        
    Returns
    -------
    int_out : int or np.nan
        A standardized integer response.
    '''
    
    # Make the input all lowercase
    str_in = str_in.lower()
    
    # Drop all whitespace
    str_in = str_in.strip()
    
    # Replace things (and then strip again afterwords)
    #  Note that the 'replace' replaces the first argument, with the second
    #   The first argument does not need to be present in the string,
    #    if it's not there 'replace' does nothing (but does not error), so the code moves on.
    str_in = str_in.replace('version', '')
    str_in = str_in.replace('python', '')
    str_in = str_in.strip()
    
    # Cast to integer, if what's left seems appropriate
    if str_in.isnumeric() and len(str_in) == 1:
        out = int(str_in)
    # Otherwise, consider input was probably ill-formed, return nan
    else: 
        out = np.nan
    
    return out

# Check how this function help standardize data:
#  Example possible answers to the question 'What is your favourite major version of Python':
print('INPUT', '\t\t-\t', 'OUTPUT')
for inp in ['version 3', '42', '2', 'python 3', 'nonsense-lolz']:
    print('{:10s} \t-\t {:1.0f}'.format(inp, example_standardize_function(inp)))

INPUT 		-	 OUTPUT
version 3  	-	 3
42         	-	 nan
2          	-	 2
python 3   	-	 3
nonsense-lolz 	-	 nan


Now we have to standardize the data!

Check all different values given for majors. It's a lot!

In [16]:
df['major'].unique()

NameError: name 'df' is not defined

We'll write a function performing some simple substring checking in order to group many responses together.

In [17]:
def standardize_major(string):
    
    string = string.lower()
    string = string.strip()
    
    if 'cog' in string:
        output = 'COGSCI'
    elif 'computer' in string:
        output = 'COMPSCI'
    elif 'cs' in string:
        output = 'COMPSCI'
    elif 'math' in string:
        output = 'MATH'
    elif 'electrical' in string:
        output = 'ECE'
    elif 'bio' in string:
        output = 'BIO'
    elif 'chem' in string:
        output = 'CHEM'
    # Otherwise, if uncaught - keep as is
    else:
        output = string
    
    return output

We then have to apply the transformation using the function we just defined.

In [18]:
df['major'] = df['major'].apply(standardize_major)

NameError: name 'df' is not defined

Previewing the results of the previous transformation.

It looks a lot better, though it's not perfect, but we'll run with this.

In [19]:
df['major'].unique()

NameError: name 'df' is not defined

### 1d) Standardize 'gender' function

Next let's check the 'gender' column. 

Check the different responses received for gender, including how many of each response we have

In [20]:
# run this to see different gender input data
df['gender'].value_counts()

NameError: name 'df' is not defined

Using a similar approach to what we used for 'major', you'll write a `standardize_gender` function.

To do this you'll:
- convert all text to lowercase
- use the string method `strip()` to remove leading and trailing characters from the gender value
- use an `if/elif/else` to:
    - output 'female' if the lowercale gender value is 'female', 'f', 'woman', 'famale',  or 'women'
    - output 'male' if the lowercase gender value is 'male', 'm', 'man', or 'men'
    - output 'nonbinary_or_trans' if the lowercase gender value is 'nonbinary' or 'transgender'
    - output `np.nan` otherwise
- return the output

In [21]:
# YOUR CODE HERE
def standardize_gender(gender):
    gender = gender.lower().strip()
    if gender in ['female', 'f', 'woman']:
        return 'female'
    elif gender in ['male', 'm', 'man']:
        return 'male'
    elif gender in ['nonbinary', 'transgender']:
        return 'nonbinary_or_trans'
    else:
        return np.nan

In [22]:
assert standardize_gender('f') == 'female'
assert standardize_gender('male') == 'male'
assert standardize_gender('Transgender') == 'nonbinary_or_trans'


### 1e) Transform 'gender' column

Apply the transformation, meaning, use your function and standardize gender in `df`

Then, drop any rows with missing gender information.

In [23]:
# YOUR CODE HERE
df['gender'] = df['gender'].apply(standardize_gender)
df.dropna(subset=['gender'], inplace=True)

NameError: name 'df' is not defined

In [24]:
# Check the results
df['gender'].unique()

NameError: name 'df' is not defined

In [25]:
assert len(df['gender'].unique()) == 3
assert df.shape == (402, 10)

NameError: name 'df' is not defined

### 1f) Unique values in `year` column
Find, programatically, the number of unique responses in the 'year' column.

Save the result in a variable named `num_unique_responses`. 

Hint: you can answer this question using the `unique` method, used above. 

In [26]:
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

In [27]:
assert num_unique_responses
assert isinstance(num_unique_responses, int)


NameError: name 'num_unique_responses' is not defined

In [28]:
# Print out all the different answers in 'year'
df['year'].unique()

NameError: name 'df' is not defined

### 1g)  Standardize 'year' column

Write a function named `standardize_year` that takes in as input a string and returns an integer.

The function will do the following (in the order specified):

Note that for these detailed instructions, each line corresponds to one line of code you need to write.

- 1) convert all characters of the string into lowercase
- 2) strip the string of all leading and trailing whitespace
- 3) replace any occurences of 'first' with '1'
- 4) replace any occurences of 'second' with '2'
- 5) replace any occurences of 'third' with '3'
- 6) replace any occurences of 'fourth' with '4'
- 7) replace any occurences of 'fifth' with '5'
- 8) replace any occurences of 'sixth' with '6'

- 9) replace any occurences of 'freshman' with '1'
- 10) replace any occurences of 'sophomore' with '2'
- 11) replace any occurences of 'junior' with '3'
- 12) replace any occurences of 'senior' with 4'
- 13) replace any occurences of 'year' with '' (remove it from the string)

- 14) replace any occurences of 'th' with '' (remove it from the string)
- 15) replace any occurences of 'rd' with '' (remove it from the string)
- 16) replace any occurences of 'nd' with '' (remove it from the string)
- 17) strip the string of all leading and trailing whitespace (again)
- 18) If the resulting string is a number and it is less than 10, then cast it into an integer and return that value
- 19) Else return np.nan to symbolize that the student's response was not a valid entry

HINTS: you will need to use the functions `lower()`, `strip()`, `isnumeric()` and `replace()`

In [29]:
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

In [30]:
assert standardize_year('2nd') == 2
assert standardize_year('sophomore') == 2
assert standardize_year('3rd year') == 3
assert standardize_year('5th') == 5
assert standardize_year('7    ') == 7
assert standardize_year('randomText') is np.nan

NameError: name 'standardize_year' is not defined

### 1h) Transform 'year' column

Use `standardize_year` to transform the data in column 'What year (in school) are you?'.

Hint: use the `apply` function AND remember to save your output inside the dataframe

In [31]:
df['year'].unique()

NameError: name 'df' is not defined

In [32]:
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

In [33]:
assert len(df['year'].unique()) == 7

NameError: name 'df' is not defined

Assuming that all is correct up to this point, the line below should show all values now found in `df['year']`. 

It should look a lot better. With this data, we can now make insightful analyses.

You should see an array with elements 1,2,3,4,5,6 and nan (not necessarily in that order).

Note that if you check the data type of this column, you'll see that pandas converts these numbers to `float`, even though the applied function returns `int`, because `np.nan` is considered a float. This is fine. 

In [34]:
df['year'].unique()

NameError: name 'df' is not defined

Let's do it again. Let's take a look at the responses in the 'weight' column, and then standardize them.

In [35]:
# First, ensure that all types are consistent, use strings
df['weight'] = df['weight'].astype(str)

NameError: name 'df' is not defined

In [36]:
# Check all the different answers we received
df['weight'].unique()

NameError: name 'df' is not defined

### 1i) Standardize 'weight' column 

Write a function named `standardize_weight` that takes in as input a string and returns an integer.

The function will do the following (in the order specified):

- 1) convert all characters of the string into lowercase
- 2) strip the string of all leading and trailing whitespace
- 3) replace any occurences of 'lbs' with '' (remove it from the string)
- 4) replace any occurences of 'lb' with '' (remove it from the string)
- 5) replace any occurences of 'pounds' with '' (remove it from the string)
- 6) If the string contains the substring 'kg', then:
    - 6.1) replace 'kg' with ''
    - 6.2) strip the string of whitespace
    - 6.3) cast the string into a float type using the function `float()`
    - 6.4) multiply the resulting float by 2.2 (an approximate conversion of kilograms to pounds) 
- 7) `try` to return the `int` of your `string`. If it cannot, return `np.nan`. 

In [37]:
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

In [38]:
assert standardize_weight('34 lbs') == 34
assert standardize_weight('101 kg') == 222


NameError: name 'standardize_weight' is not defined

### 1j) Transform 'weight' column 

Use `standardize_weight` to transform the data in the 'weight' column.

Hint: use the `apply` function AND remember to save your output inside the dataframe

In [39]:
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

In [40]:
assert df['weight'].unique().shape == (83,)

NameError: name 'df' is not defined

Now, let's see the result of our hard work. The code below should output all numbers (or nan).

In [41]:
df['weight'].unique()

NameError: name 'df' is not defined

So far, you've gotten a taste of what it is like to deal with messy data. It's not easy, as you can tell. 

The last variable we need to standardize for the purposes of our analysis is 'height'. We will standardize that one for you.

Do read the code below and try to understand what it is doing.

In [42]:
# First, we'll look at the possible values for height
df['height'].unique()

NameError: name 'df' is not defined

It seems like we'll have to handle different measurement systems. Ugh, ok...


Let's write a function that converts all those values to inches.

In [43]:
# convert all values to inches
def standardize_height(string):
    
    orig = string
    output = None
    
    # Basic string pre-processing
    string = string.lower()
    string = string.strip()
    
    string = string.replace('foot', 'ft')
    string = string.replace('feet', 'ft')
    string = string.replace('inches', 'in')
    string = string.replace('inch', 'in')
    string = string.replace('meters', 'm')
    string = string.replace('meter', 'm')
    string = string.replace('centimeters', 'cm')
    string = string.replace('centimeter', 'cm')
    string = string.replace(',', '')
    string = string.strip()
    
    # CASE 1: string is written in the format FEET <DIVIDER> INCHES
    dividers = ["'", "ft", "’", '”', '"','`', "-", "''"]
    
    for divider in dividers:
        
        # Split it into its elements
        elements = string.split(divider)

        # If the divider creates two elements
        if (len(elements) >= 2) and ((len(string) -1) != string.find(divider)):
            feet = elements[0]
            inch = elements[1] if elements[1] is not '' else '0'
            
            # Cleaning extranious symbols
            for symbol in dividers:
                feet = feet.replace(symbol, '')
                inch = inch.replace(symbol, '')
                inch = inch.replace('in','')
            
            # Removing whitespace
            feet = feet.strip()
            inch = inch.strip()
            
            # By this point, we expect 'feet' and 'inch' to be numeric
            # If not...we ignore this case
            if feet.replace('.', '').isnumeric() and inch.replace('.', '').isnumeric():
                
                # Converting feet to inches and adding it to the current inches
                output = (float(feet) * 12) + float(inch)
                break
            
    # CASE 2: string is written in the format FEET ft INCHES in 
    if ('ft' in string) and ('in' in string):
        
        # Split it into its elements
        elements = string.split('ft')
        feet = elements[0]
        inch = elements[1]
        
        # Removing extraneous symbols and stripping whitespace
        inch = inch.replace('inch', '')
        inch = inch.replace('in', '')
        feet = feet.strip()
        inch = inch.strip()
        
        # By this point, we expect 'feet' and 'inch' to be numeric
        # If not...we ignore this case
        if feet.replace('.', '').isnumeric() and inch.replace('.', '').isnumeric():
                
            # Converting feet to inches and adding it to the current inches
            output = (float(feet) * 12) + float(inch)
        
    # CASE 3: answer was given ONLY in cm
    #  Convert to inches: approximately 0.39 inches in a meter
    elif 'cm' in string:
        centimeters = string.replace('cm', '')
        centimeters = centimeters.strip()
        
        if centimeters.replace('.', '').isnumeric():
            output = float(centimeters) * 0.39
        
    # CASE 4: answer was given ONLY in meters
    #  Convert to inches: approximately 39 inches in a meter
    elif 'm' in string:
        
        meters = string.replace('m', '')
        meters = meters.strip()
        
        if meters.replace('.', '').isnumeric():
            output = float(meters)*39
        
    # CASE 5: answer was given ONLY in feet
    elif 'ft' in string:

        feet = string.replace('ft', '')
        feet = feet.strip()
        
        if feet.replace('.', '').isnumeric():
            output = float(feet)*12
    
    # CASE 6: answer was given ONLY in inches
    elif 'in' in string:
        inches = string.replace('in', '')
        inches = inches.strip()
        
        if inches.replace('.', '').isnumeric():
            output = float(inches)
        
    # CASE 7: answer not covered by existing scenarios / was invalid. 
    #  Return NaN
    if not output:
        output = np.nan

    return output

In [44]:
# Applying the transformation and dropping invalid rows
df['height'] = df['height'].apply(standardize_height)
df = df.dropna(subset=['height'])

NameError: name 'df' is not defined

In [45]:
# Check the height data, after applying our standardization
df['height'].unique()

NameError: name 'df' is not defined

In [46]:
# Ensuring that the data types are correct - type cast age to int.
df['age'] = df['age'].astype(np.int64)

# Check out the data, after we've cleaned it!
df.head()

NameError: name 'df' is not defined

In [47]:
# Check that the dataframe has the right number of rows
#  If this doesn't pass - check your code in the section above.
assert len(df) == 365

NameError: name 'df' is not defined

## Part 2: Exploratory Data Vizualization (0.8 points)

First, we need to do some exploratory data visualization, to get a feel for the data. 

For plotting questions, do not change or move the `plt.gcf()` lines. 

### 2a) Plot the data

Using `scatter_matrix`, from `pandas`' `plotting` module to plot `df`. Assign it to a variable called `fig`.


In [48]:
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

In [49]:
assert np.all(fig)

NameError: name 'fig' is not defined

### 2b) Plot a bar chart showing the number of students in each major. 

Hint: 
- if using `seaborn`, you're looking to make a countplot
- if using pandas, you can use `value_counts` to get the counts for each major. You can then use the `plot` method from `pandas` for plotting (You don't need `matplotlib`). 


In [50]:
# YOUR CODE HERE
raise NotImplementedError()

f1 = plt.gcf()

NotImplementedError: 

In [51]:
assert f1.gca().has_data()

NameError: name 'f1' is not defined

### 2c) Plot a histogram of the height data for all students who wrote 'COGSCI' as their major.


In [52]:
# YOUR CODE HERE
raise NotImplementedError()

f2 = plt.gcf()

NotImplementedError: 

In [53]:
assert f2.gca().has_data()

NameError: name 'f2' is not defined

### 2d) Plot a histogram of the height data for all students who wrote 'COMPSCI' as their major.


In [54]:
# YOUR CODE HERE
raise NotImplementedError()

f3 = plt.gcf()

NotImplementedError: 

In [55]:
assert f3.gca().has_data()

NameError: name 'f3' is not defined

## Part 3: Exploring The Data (0.4 points)

Beyond just plotting the data, we should check some other basic properties of the data. This serves both as a way to get a 'feel' for the data, and to look for any quirks or oddities about the data, that may indicate issues that need resolving. To do this, let's explore that data a bit (not limiting ourselves to only features that we plan to use - exploring the dataset as a whole can help us find any issues). 

Notes:
- Your answers should NOT be pandas objects (Series or DataFrames), extract answers so the variables are ints, floats or strings (as appropriate).
- You must answer these questions programmatically: do not count / check and hard code particular values.

### 3a) How many different majors are in the dataset?

Calculate this programmatically, and save this number to a variable `n_majors`.

In [56]:
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

In [57]:
assert n_majors > 0 and n_majors < 25


NameError: name 'n_majors' is not defined

### 3b) What is the range (max value - min value) of ages in the dataset?

Calculate this programmatically and save this number to a variable `r_age`.

In [58]:
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

In [59]:
assert r_age > 0 and r_age < 50 


NameError: name 'r_age' is not defined

## Part 4: Data Analysis (4.15 points)

Now let's analyze the data, to address our research question.

For the purposes of this analysis, let's assume we need at least 75 students per major to analyze the height data. 

This means we are only going to use data from people who wrote 'COGSCI' or 'COMPSCI' as their major. 


### 4a) Pull out the data we are going to use:

Save the height data for all 'COGSCI' majors to a variable called `h_co`

Save the height data for all 'COMPSCI' majors to a variable called `h_cs`

In [60]:
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

In [61]:
assert np.all(h_co)
assert np.all(h_cs)

assert len(h_co) == 178
assert len(h_cs) == 164

NameError: name 'h_co' is not defined

### 4b) What is the average (mean) height for students from each major?

Calulate this programmatically and save these values to `avg_h_co` for COGSCI students, and `avg_h_cs` for COMPSCI students. 


In [62]:
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

In [63]:
assert avg_h_co
assert avg_h_cs


NameError: name 'avg_h_co' is not defined

In [64]:
# Print out the average heights - this code provided
print('Average height of cogs majors is \t {:2.2f} inches'.format(avg_h_co))
print('Average height of cs majors is \t\t {:2.2f} inches'.format(avg_h_cs))

NameError: name 'avg_h_co' is not defined

Based on the cell above, it looks like there might indeed be a difference in the average height for students in cogs vs cs majors. 

Now we want to statistically test this difference. To do so, we will use a t-test. 

### 4c) Compare distributions: t-test 

Use the `ttest_ind` function) to compare the two height distributions (`h_co` vs `h_cs`)

`ttest_ind` returns a t-statistic, and a p-value. Save these outputs to `t_val` and `p_val` respectively. 


In [65]:
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

In [66]:
assert t_val
assert p_val


NameError: name 't_val' is not defined

In [67]:
# Check if statistical test passes significance, using an alpha value of 0.01. This code provided.
if p_val < 0.01:
    print('Data Science accomplished, there is a significant difference!')
else:
    print('There is NOT a significant difference!')
    
# Editorial note:
#  Chasing significant p-values as the goal itself is not actually a good way to do data (or any) science :)

NameError: name 'p_val' is not defined

Note: this test *should* pass significance. If it doesn't, double check your code up until this point.

So - we've reached a conclusion! We're done right!?

Nope. We have a first pass analysis, and an interim conclusion that happens to follow our hypothesis. 

Now let's try to break it. 

#### Let's explore some more

You should always interrogate your findings, however they come out. What could be some alternate explanations, that would change our interpretations of the current analysis?

In this case, we should be worried about confounding variables. We want to be able to say whether height relates to major specifically, but it could be the case that some other variable, that happens to differ between majors, better explains the differences in height. 

In this case, we also have data on gender. Let's check if differences in the gender ratio of the two majors can explain the difference in height. 

### 4d) Digging Deeper

Using `value_counts` from pandas, extract the number of 'male', 'female', and 'nonbinary_or_trans' individuals, separately for cogs and cs students. 

To do so:

- select from the `df` each major, separately, extract the gender column, and use the `value_counts` method.
- Save the counts for each gender for 'COGSCI' majors to a variable called `g_co`
-  Save the counts for each gender for 'COMPSCI' majors to a variable called `g_cs`


In [68]:
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

In [69]:
assert np.all(g_co)
assert np.all(g_cs)

assert g_co[0] == 91
assert g_cs[1] == 38
assert g_cs[2] == 1

NameError: name 'g_co' is not defined

### 4e) What is the ratio of women in each major? 

By ratio, we mean the proportion of students that are female, as a ratio. This will be a value between 0.0 and 1.0, calculated as #F / (#F + #M + #nonbinary_or_other) - done separately for each major

You can use the `g_co` and `g_cs` variables to calculate these.

Save the ratio of women in COGSCI to a variable `r_co`.

Save the ratio of women in COMPSCI to a variable `r_cs`.

Note: keep these numbers as ratios (they should be decimal numbers, less than 1).


In [70]:
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

In [71]:
assert r_co
assert r_cs


NameError: name 'r_co' is not defined

Make sure you print out and check the values of these ratios. They seem pretty different.

We can actually ask, using a chi-squared test, whether this difference in gender-ratio between the majors is signficantly different.

Code to do this is provided below. 

Run a chi-squared test of the difference of ratios of categorical data between groups:


In [72]:
chisq, p_val_chi = stats.chisquare(np.array([g_co.values, g_cs.values]), axis=None)

if p_val_chi < 0.01:
    print('There is a significant difference in ratios!')

NameError: name 'stats' is not defined

### 4f) Subsetting data

Create a new dataframe, called `df2`, which only includes data from 'COGSCI' and 'COMPSCI' majors.

Hint: you can do this using the or operater '|', with loc. 


In [73]:
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

In [74]:
assert isinstance(df2, pd.DataFrame)
assert df2.shape == (342, 10)
assert set(df2['major']) == set(['COGSCI', 'COMPSCI'])

NameError: name 'df2' is not defined

### 4g) Pivot Tables

Another way to look at these kinds of comparisons is pivot tables. 

Use the pandas `pivot_table` method to create a pivot table, assign it to a variable `pv`.

Set the values as 'height', and the indices as 'gender' and 'major' in the pivot table.

Make sure you do this using `df2`. 


In [75]:
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

In [76]:
assert np.all(pv)
assert isinstance(pv.index, pd.MultiIndex)


NameError: name 'pv' is not defined

Print out the pivot table you just created. 

Compare the average height values, split up by major and gender.

Does it look like there are differences in heights by major, when spit up by gender?


In [77]:
pv

NameError: name 'pv' is not defined

Let's recap where we are:
- Our initial hypothesis suggested there is a significant difference between heights of people in different majors. 
- However, further analyses suggested there may be a confounding variable, as there is also a significantly different gender balance between majors. 

Checking the average height, per major, split up by gender, suggests there may not be a difference between major, other than what is explained by gender. 

Now we want to statistically ask this question: is there still a difference in height between majors, when controlling for differences in gender?

#### Linear Models

For the following question you will need to make some linear models, using Ordinary Least Squares (OLS).

There is more than one way to do this in Python. For the purposes of this assignment, you must use the method that is outlined in the 'LinearModels' Tutorial, using patsy, and statsmodels. 

That is: 
- Create design matrices with `patsy.dmatrices`
- Initialize an OLS model with `sm.OLS`
- Fit the OLS model
- Check the summary for results. 

### 4h) Linear model

Create a linear model to predict height from major (using `df2` as data).

Use `patsy.dmatrices` to create the design matrices, calling the outputs `outcome_1`, `predictors_1`.

Create an OLS model (`sm.OLS`) using `outcome_1` and `predictors_1`. Call it `mod_1`.

Fit the model, assigning it to `res_1`.

In [78]:
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

In [79]:
assert isinstance(outcome_1, patsy.design_info.DesignMatrix)
assert isinstance(predictors_1, patsy.design_info.DesignMatrix)
assert isinstance(mod_1, sm.regression.linear_model.OLS)
assert isinstance(res_1, sm.regression.linear_model.RegressionResultsWrapper)


NameError: name 'outcome_1' is not defined

In [80]:
# Print out the summary results of the model fitting
print(res_1.summary())

NameError: name 'res_1' is not defined

### 4i) Assess significance: `mod_1`

Based on the model you ran above (using alpha value of 0.01), does major significantly predict height?

Set your answer as a boolean (True / False) to a variable called `lm_1`.

In [81]:
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

In [82]:
assert isinstance(lm_1, bool)


NameError: name 'lm_1' is not defined

### 4j) Multivariate regression

Create a linear model to predict height from both major and gender (using `df2` as data).

Use `patsy.dmatrices` to create the design matrices, calling the outputs `outcome_2`, `predictors_2`

Create an OLS model (`sm.OLS`) using `outcome_2` and `predictors_2`.  Call it `mod_2`.

Fit the model, assigning it to `res_2`.


In [83]:
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

In [84]:
assert isinstance(outcome_2, patsy.design_info.DesignMatrix)
assert isinstance(predictors_2, patsy.design_info.DesignMatrix)
assert isinstance(mod_2, sm.regression.linear_model.OLS)
assert isinstance(res_2, sm.regression.linear_model.RegressionResultsWrapper)


NameError: name 'outcome_2' is not defined

In [85]:
# Print out the results 
print(res_2.summary())

NameError: name 'res_2' is not defined

### 4k) Assess significance: `mod_2`

Based on the model you ran above (using alpha value of 0.01), does major significantly predict height?

Set your answer as a boolean (True / False) to a variable called `lm_2`

In [86]:
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

In [87]:
assert isinstance(lm_2, bool)


NameError: name 'lm_2' is not defined

## Part 5: Discussion & Conclusions (0.15 points)

### 5a) Conclusion

Set a boolean variable, called `ans`, as True or False as the answer to the following statement:

We have evidence supporting our research hypothesis:

> People in different majors have systematically different heights (and this difference can be tied to their major).


In [88]:
# YOUR CODE HERE
raise NotImplementedError()

NotImplementedError: 

In [89]:
assert isinstance(ans, bool)


NameError: name 'ans' is not defined

### 5b) Summary

Write a short response (1-2 sentence) summarizing the results.

Did we support our hypothesis? Why or why not? What turned out to be the finding(s)?

YOUR ANSWER HERE

## The End!

Good work! Have a look back over your answers, and also make sure to `Restart & Run All` from the kernel menu to double check that everything is working properly. You can also use the 'Validate' button above, which runs your notebook from top to bottom and checks to ensure all `assert` statements pass silently. When you are ready, submit on datahub!